In [1]:
%env JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64
%env PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin

env: JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64
env: PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin


In [2]:
import findspark
import pandas as pd
import numpy as np

import pyspark as psk
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as f
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import DataFrame
from pyspark.sql.functions import *

import time as t 
from datetime import date, datetime, timedelta
import joblib
from joblib import Parallel, delayed

import multiprocessing
from functools import reduce  
from math import modf
import sqlite3
import os
import sys

from delta import * 
from delta.tables import *
from delta import configure_spark_with_delta_pip
# import covalent as ct

import subprocess
import re
import random

from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
# import umap.umap_ as umap
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


findspark.init()
findspark.find()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# os.environ['PYDEVD_DISABLE_FILE_VALIDATION']=1

spark = SparkSession.builder \
       .master("local[2]") \
       .appName("MyApp") \
       .config("spark.driver.maxResultSize", "20g")\
       .config("spark.driver.memory", "100g")\
       .getOrCreate()
sc = spark.sparkContext

# sqlContext = SQLContext(sc)


#both works
# 1: 
# spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")    #To resolve the error for p1075_38 to_timestamp formating: You may get a different result due to the upgrading to Spark >= 3.0: Fail to parse '1/2/2019 20:40:00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
# Set Spark SQL legacy time parser policy to LEGACY to handle older date formats
# 2:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
# Increase the max fields in the string representation of a plan
spark.conf.set("spark.sql.execution.arrow.enabled", "false")
spark.conf.set("spark.sql.debug.maxToStringFields", 2000)

ps: /storage/icds/RISE/sw8/anaconda/anaconda3/envs/tensorflow/lib/libuuid.so.1: no version information available (required by /usr/lib64/libblkid.so.1)
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/03 11:11:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/03 11:11:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/04/03 11:11:15 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


In [3]:
spark.read.option("header",True) \
          .csv("/storage/home/yqf5148/work/volvoPennState/data/dataset/VINs_data.csv") \
          .createOrReplaceTempView("VINs_data")

spark.read.option("header",True) \
          .csv("/storage/home/yqf5148/work/volvoPennState/PopulationWithChassisId.csv") \
          .createOrReplaceTempView("population")

In [4]:
df_all_pop = spark.sql('select * from population')
df_all_pop_p = df_all_pop.toPandas()

In [5]:
# Get all column names
all_columns = df_all_pop.columns

# Print the first 10
print("First 10 column names:")
for col in all_columns[:11]:
    print(col)


First 10 column names:
VIN
CHASSIS_ID
ENGINE_ASSEMB_DATE
ENGINE_ASSEMB_MONTH
VEH_ASSEMB_DATE
VEH_ASSEMB_MONTH
INS_DATE
INS_MONTH
ENGINE_SIZE
ENGINE_HP
VEH_TYPE


In [6]:
# Get all column names
all_columns = df_all_pop.columns

vin_column = [all_columns[0]]

# Include column 2 (index 1)
extra_column = [all_columns[2]]

# Pick columns from index 8 to the end
main_columns = all_columns[8:]

# Combine them
selected_columns = vin_column + extra_column + main_columns

# Print how many columns are selected
print(f"Number of selected columns: {len(selected_columns)}")

# Print their names
print("Selected column names:")
for col in selected_columns:
    print(col)


Number of selected columns: 593
Selected column names:
VIN
ENGINE_ASSEMB_DATE
ENGINE_SIZE
ENGINE_HP
VEH_TYPE
_KOLA_01X
_KOLA_02X
_KOLA_03X
_KOLA_04X
_KOLA_05X
_KOLA_06X
_KOLA_07X
_KOLA_08A
_KOLA_08X
_KOLA_09X
_KOLA_0AX
_KOLA_0BX
_KOLA_0CX
_KOLA_0DX
_KOLA_0EA
_KOLA_0EX
_KOLA_0GX
_KOLA_0HA
_KOLA_0HX
_KOLA_0JX
_KOLA_0KX
_KOLA_0LX
_KOLA_0NX
_KOLA_0PA
_KOLA_0PX
_KOLA_0RA
_KOLA_0RX
_KOLA_0SX
_KOLA_15X
_KOLA_19A
_KOLA_1BX
_KOLA_1CX
_KOLA_1DA
_KOLA_1DX
_KOLA_1EA
_KOLA_1GA
_KOLA_1HA
_KOLA_1JX
_KOLA_1KX
_KOLA_1LA
_KOLA_1NX
_KOLA_1PB
_KOLA_1PX
_KOLA_1QB
_KOLA_1RB
_KOLA_1RX
_KOLA_1SB
_KOLA_1SX
_KOLA_1TX
_KOLA_1UX
_KOLA_1VX
_KOLA_1YA
_KOLA_1ZC
_KOLA_20X
_KOLA_21X
_KOLA_24X
_KOLA_26X
_KOLA_28X
_KOLA_2AC
_KOLA_2BC
_KOLA_2DX
_KOLA_2JC
_KOLA_2KX
_KOLA_2MC
_KOLA_2PB
_KOLA_2PX
_KOLA_2WX
_KOLA_2XX
_KOLA_31X
_KOLA_34X
_KOLA_36X
_KOLA_38X
_KOLA_39X
_KOLA_3AX
_KOLA_3FC
_KOLA_3GC
_KOLA_3GX
_KOLA_3HA
_KOLA_3HX
_KOLA_3IA
_KOLA_3MB
_KOLA_3NC
_KOLA_3PX
_KOLA_3RC
_KOLA_3RX
_KOLA_3UC
_KOLA_3ZX
_KOLA_40X
_KOLA_41X
_

In [7]:
# # Convert Spark DataFrame to Pandas (already done as df_all_pop_p)
# vin_column = "VIN"
# df = df_all_pop_p.copy()

# # Step 1: Identify KOLA and non-KOLA columns
# kola_columns = [col for col in selected_columns if "KOLA" in col]
# non_kola_columns = [col for col in selected_columns if col not in kola_columns or col == vin_column]

# # Step 2: Drop NA rows (optional, depending on your data)
# df_filtered = df[non_kola_columns + kola_columns].dropna()

# # Step 3: One-hot encode KOLA categorical columns
# encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
# kola_encoded = encoder.fit_transform(df_filtered[kola_columns])

# # Step 4: Apply PCA
# pca = PCA(n_components=10)  # Tune this number if needed
# kola_pca = pca.fit_transform(kola_encoded)

# # Step 5: Create DataFrame with PCA features
# pca_columns = [f"KOLA_PCA_{i+1}" for i in range(kola_pca.shape[1])]
# df_kola_pca = pd.DataFrame(kola_pca, columns=pca_columns, index=df_filtered.index)

# # Step 6: Concatenate non-KOLA features with reduced KOLA features
# df_final = pd.concat([df_filtered[non_kola_columns], df_kola_pca], axis=1)

# # Show result
# print("Final shape:", df_final.shape)
# print("Final columns:", df_final.columns.tolist())

In [8]:
# vin_column = "VIN"
# df = df_all_pop_p.copy()

# # Step 1: Identify KOLA and non-KOLA columns
# kola_columns = [col for col in selected_columns if "KOLA" in col]
# non_kola_columns = [col for col in selected_columns if col not in kola_columns or col == vin_column]

# # Step 2: Drop NA rows
# df_filtered = df[non_kola_columns + kola_columns].dropna()

# # Step 3: One-hot encode KOLA categorical columns
# encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
# kola_encoded = encoder.fit_transform(df_filtered[kola_columns])
# encoded_feature_names = encoder.get_feature_names_out(kola_columns)

# # Step 4: Apply PCA to retain 95% of variance
# pca = PCA(n_components=0.95, svd_solver='full')
# kola_pca = pca.fit_transform(kola_encoded)

# # Step 5: Create DataFrame with PCA features
# pca_columns = [f"KOLA_PCA_{i+1}" for i in range(kola_pca.shape[1])]
# df_kola_pca = pd.DataFrame(kola_pca, columns=pca_columns, index=df_filtered.index)

# # Step 6: Combine with non-KOLA features
# df_final = pd.concat([df_filtered[non_kola_columns], df_kola_pca], axis=1)

# # Step 7: Save encoder and PCA
# joblib.dump(encoder, "kola_encoder.joblib")
# joblib.dump(pca, "kola_pca_model.joblib")
# df_final.to_csv("population_with_reduced_kola.csv", index=False)

# # Step 8: Show top contributing KOLA features per PCA component
# components_df = pd.DataFrame(pca.components_, columns=encoded_feature_names)

# print("\n Top contributing original KOLA features per PCA component:")
# for i in range(components_df.shape[0]):
#     print(f"\n PCA Component {i+1}:")
#     print(components_df.iloc[i].abs().sort_values(ascending=False).head(5))

# # Final summary
# print(f"\n Final shape: {df_final.shape}")
# print(f" Explained Variance Ratio (Total Retained): {pca.explained_variance_ratio_.sum():.2%}")


In [9]:
# def apply_pca_then_umap(df: pd.DataFrame, kola_columns: list, non_kola_columns: list,
#                         pca_components=100, umap_components=10, random_state=42):
#     """
#     Applies PCA followed by UMAP on one-hot encoded KOLA features to reduce high dimensionality.
#     Combines reduced components with non-KOLA features.
#     """
#     print("Step 1: Dropping NA rows from selected columns...")
#     try:
#         df_filtered = df[non_kola_columns + kola_columns].dropna()
#         print(f"Step 1 complete: Filtered data shape is {df_filtered.shape}")
#     except Exception as e:
#         print("Error in Step 1 (Dropping NA rows):", e)
#         return None

#     print("Step 2: One-hot encoding KOLA columns...")
#     encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
#     try:
#         kola_encoded = encoder.fit_transform(df_filtered[kola_columns])
#         print(f"Step 2 complete: One-hot encoded shape is {kola_encoded.shape}")
#     except Exception as e:
#         print("Error in Step 2 (One-hot encoding):", e)
#         return None

#     print(f"Step 3: Applying PCA to reduce to {pca_components} components...")
#     try:
#         pca = PCA(n_components=pca_components, random_state=random_state)
#         kola_pca = pca.fit_transform(kola_encoded)
#         print(f"Step 3 complete: PCA output shape is {kola_pca.shape}")
#     except Exception as e:
#         print("Error in Step 3 (PCA):", e)
#         return None

#     print(f"Step 4: Applying UMAP to reduce to {umap_components} dimensions...")
#     try:
#         reducer = umap.UMAP(n_components=umap_components, random_state=random_state)
#         kola_umap = reducer.fit_transform(kola_pca)
#         print(f"Step 4 complete: UMAP output shape is {kola_umap.shape}")
#     except Exception as e:
#         print("Error in Step 4 (UMAP):", e)
#         return None

#     print("Step 5: Creating UMAP DataFrame and concatenating with non-KOLA features...")
#     try:
#         umap_columns = [f"KOLA_UMAP_{i+1}" for i in range(umap_components)]
#         df_kola_umap = pd.DataFrame(kola_umap, columns=umap_columns, index=df_filtered.index)
#         df_final = pd.concat([df_filtered[non_kola_columns], df_kola_umap], axis=1)
#         print(f"Step 5 complete: Final dataset shape is {df_final.shape}")
#     except Exception as e:
#         print("Error in Step 5 (Concatenation):", e)
#         return None

#     print("Dimensionality reduction using PCA followed by UMAP completed successfully.")
#     return df_final


In [10]:
# # Step 1: Identify KOLA and non-KOLA columns
# kola_columns = [col for col in selected_columns if "KOLA" in col]
# non_kola_columns = [col for col in selected_columns if col not in kola_columns or col == vin_column]


# df_umap_result = apply_pca_then_umap(df, kola_columns, non_kola_columns)

In [11]:
# def apply_kmeans_clustering(df_reduced: pd.DataFrame, n_clusters=10, random_state=42):
#     """
#     Applies KMeans clustering to the PCA-reduced dataset and returns a DataFrame with cluster labels.
#     Step-by-step prints are included to show progress.
#     """
#     print("Step 1: Starting KMeans clustering process...")
#     print(f"    - Number of clusters: {n_clusters}")
#     print(f"    - Input data shape: {df_reduced.shape}")

#     try:
#         print("Step 2: Initializing KMeans model...")
#         kmeans = KMeans(n_clusters=n_clusters, random_state=random_state)

#         print("Step 3: Fitting the KMeans model and predicting clusters...")
#         cluster_labels = kmeans.fit_predict(df_reduced)
#         print("    - KMeans clustering complete.")

#         print("Step 4: Adding cluster labels to the DataFrame...")
#         df_clustered = df_reduced.copy()
#         df_clustered["CLUSTER"] = cluster_labels
#         print("    - Cluster labels added.")

#         print("Step 5: Calculating silhouette score to evaluate clustering...")
#         score = silhouette_score(df_reduced, cluster_labels)
#         print(f"    - Silhouette Score for {n_clusters} clusters: {score:.4f}")

#         print("Clustering process completed successfully.")
#         return df_clustered

#     except Exception as e:
#         print("Error during KMeans clustering process:", e)
#         return None


In [12]:
# # Make sure you only pass PCA-reduced KOLA feature columns
# pca_columns = [col for col in df_final.columns if col.startswith("KOLA_PCA_")]
# df_kmeans_result = apply_kmeans_clustering(df_final[pca_columns], n_clusters=10)

In [13]:
def evaluate_pca_component_range(df, categorical_columns, non_kola_columns, k=10):
    print("🔹 Step 1: Dropping NA rows...")
    df_filtered = df[non_kola_columns + categorical_columns].dropna()
    print(f"    - Filtered shape: {df_filtered.shape}")

    print("🔹 Step 2: One-hot encoding KOLA features...")
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    kola_encoded = encoder.fit_transform(df_filtered[categorical_columns])
    print(f"    - Encoded shape: {kola_encoded.shape}")

    silhouette_scores = []

    for n_components in range(6, 16):
        print(f"\n🔸 Testing PCA with {n_components} components...")
        pca = PCA(n_components=n_components)
        pca_result = pca.fit_transform(kola_encoded)

        print("    - Clustering with KMeans...")
        kmeans = KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(pca_result)

        score = silhouette_score(pca_result, labels)
        silhouette_scores.append(score)
        print(f"    - Silhouette Score: {score:.4f}")

    print("\n🔹 Plotting silhouette scores...")
    plt.figure(figsize=(10, 6))
    plt.plot(range(6, 16), silhouette_scores, marker='o')
    plt.title("Silhouette Score vs. Number of PCA Components")
    plt.xlabel("Number of PCA Components")
    plt.ylabel("Silhouette Score")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [14]:
vin_column = "VIN"
df = df_all_pop_p.copy()

# Assuming df is your full dataset and selected_columns is already defined
kola_columns = [col for col in selected_columns if "KOLA" in col]
non_kola_columns = [col for col in selected_columns if col not in kola_columns or col == vin_column]

evaluate_pca_component_range(df, categorical_columns=kola_columns, non_kola_columns=non_kola_columns, k=10)


🔹 Step 1: Dropping NA rows...
    - Filtered shape: (321746, 593)
🔹 Step 2: One-hot encoding KOLA features...
    - Encoded shape: (321746, 7845)

🔸 Testing PCA with 6 components...
    - Clustering with KMeans...
    - Silhouette Score: 0.5588

🔸 Testing PCA with 7 components...
    - Clustering with KMeans...
    - Silhouette Score: 0.5404

🔸 Testing PCA with 8 components...
    - Clustering with KMeans...
    - Silhouette Score: 0.4887

🔸 Testing PCA with 9 components...
    - Clustering with KMeans...
    - Silhouette Score: 0.4544

🔸 Testing PCA with 10 components...
    - Clustering with KMeans...
    - Silhouette Score: 0.4422

🔸 Testing PCA with 11 components...
    - Clustering with KMeans...
    - Silhouette Score: 0.4239

🔸 Testing PCA with 12 components...
    - Clustering with KMeans...
    - Silhouette Score: 0.4053

🔸 Testing PCA with 13 components...
    - Clustering with KMeans...
    - Silhouette Score: 0.3762

🔸 Testing PCA with 14 components...
    - Clustering with

NameError: name 'plt' is not defined